In [ ]:
# BOILERPLATE TO WHAT IMAGINED TO BE A ROADTRIP ADVISOR. INPUT MODE OF TRAVEL, START AND DESTINY, A MARGIN HOW FAR OF THE IDEAL TRACK YOU WANT TO VENTURE (AND THE TYPE OF SPOTS YOU ARE INTERESTED IN) AND GET ACCORDING MAP RETURNED. 


In [ ]:
import googlemaps
from googlemaps import convert
from googlemaps.convert import latlng
import requests
import json

from shapely.geometry import Point, Polygon


waypoints = []

def initMap():
    global maps, waypoints, polygon_coords, api_key, poly
    api_key = 'GOOGLE MAPS API KEY'  # Replace with your own API key
    gmaps = googlemaps.Client(key=api_key)

    directions_result = gmaps.directions("München", "Starnbergersee", mode="driving")
    directions = directions_result[0]['overview_polyline']['points']
    waypoints = convert.decode_polyline(directions)

    polygon_coords = polygon_points(waypoints)
    
    l = []
    for i in polygon_coords: 
        l.append((i['lat'], i['lng']))
        
    poly = Polygon(l)
    
    results = []
    for j in range(0, len(waypoints), 80):
        location = (waypoints[j]['lat'], waypoints[j]['lng'])
        nearby_places = nearby_search(location, radius=5000, typ='gas_station')
        
        for result in range(0, len(nearby_places)):
            pt = Point(nearby_places[result]['location'])
            if poly.contains(pt):
                #googlemaps.geometry.contains_location(result.geometry.location, polygon_bound)
                results.append(nearby_places[result])
                
    return results


def nearby_search(location, radius, typ): 
    
    global response
    
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location[0]}%2C{location[1]}&radius={radius}&type={typ}&keyword=cruise&key={api_key}"

    payload = {}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    places = []

    for i in response.json()['results']:
        if i['business_status'] == 'OPERATIONAL':
            nlocation = (i['geometry']['location']['lat'], i['geometry']['location']['lng'])
            name = i['name']
            types = i['types']

            places.append({'location': nlocation, 'name': name, 'types': types})

    return places
    
def polygon_points(waypoints):

    poly_points = waypoints
    poly_length = len(poly_points)
    
    upper_bound = []
    lower_bound = []

    for j in range(poly_length):
        new_points = polygon_array(poly_points[j]['lat'])
        upper_bound.append({"lat": new_points[0], "lng": poly_points[j]['lng']})
        lower_bound.append({"lat": new_points[1], "lng": poly_points[j]['lng']})

    reverse_bound = lower_bound[::-1]
    full_poly = upper_bound + reverse_bound
    return full_poly

def polygon_array(latitude):
    R = 6378137
    pi = 3.14
    upper_offset = 1000
    lower_offset = -1000

    lat_up = upper_offset / R
    lat_down = lower_offset / R

    lat_upper = latitude + (lat_up * 180) / pi
    lat_lower = latitude + (lat_down * 180) / pi

    return [lat_upper, lat_lower]

r = initMap()


In [ ]:
r